# Image Captioning with Deep Learning - Training Notebook

This notebook trains both RNN and Transformer models for automated image captioning.

**Expected Training Time:** ~2-3 hours with free Tesla T4 GPU

## Quick Start
1. Click **Runtime** → **Change runtime type** → Select **T4 GPU**
2. Click **Runtime** → **Run all** (or press Ctrl+F9)
3. Wait for training to complete
4. Download trained models from the Files panel

---

## Step 1: Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Please enable GPU: Runtime → Change runtime type → T4 GPU")

In [ ]:
# Install required packages
!pip install -q transformers pillow tqdm nltk matplotlib seaborn gradio
!pip install -q git+https://github.com/jmhessel/pycocoevalcap.git

print("✅ All packages installed!")

## Step 2: Clone Repository and Download Dataset

In [ ]:
# Clone the repository
!git clone https://github.com/yichi0812/image-captioning-neural-networks.git
%cd image-captioning-neural-networks

print("✅ Repository cloned!")

In [ ]:
# Download Flickr8k dataset from Kaggle
# You need to upload your kaggle.json file first

from google.colab import files
import os

print("📤 Please upload your kaggle.json file (from https://www.kaggle.com/settings/account)")
print("   If you don't have it: Go to Kaggle → Account → Create New API Token")
uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle credentials configured!")

In [ ]:
# Download Flickr8k dataset
!pip install -q kaggle
!kaggle datasets download -d adityajn105/flickr8k
!unzip -q flickr8k.zip -d data/
!mv data/Images data/images

print("✅ Dataset downloaded and extracted!")
print(f"Total images: {len(os.listdir('data/images'))}")

## Step 3: Prepare Dataset

In [ ]:
# Prepare train/val/test splits
!python src/prepare_data.py --dataset flickr8k --data-dir ./data

print("✅ Dataset preparation complete!")

## Step 4: Train RNN Model

Training RNN with attention mechanism (~1 hour on T4 GPU)

In [ ]:
import time
start_time = time.time()

# Train RNN model
!python src/train.py --model rnn --epochs 20 --batch-size 64

elapsed = (time.time() - start_time) / 60
print(f"\n✅ RNN training complete! Time: {elapsed:.1f} minutes")

## Step 5: Train Transformer Model

Training Transformer decoder (~1.5 hours on T4 GPU)

In [ ]:
start_time = time.time()

# Train Transformer model
!python src/train.py --model transformer --epochs 20 --batch-size 64

elapsed = (time.time() - start_time) / 60
print(f"\n✅ Transformer training complete! Time: {elapsed:.1f} minutes")

## Step 6: Evaluate Models

In [ ]:
# Evaluate both models and compare
!python src/run_evaluation.py

# Display results
import json
with open('outputs/evaluation_results.json', 'r') as f:
    results = json.load(f)

print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
for model_name, metrics in results.items():
    print(f"\n{model_name.upper()}:")
    for metric, score in metrics.items():
        print(f"  {metric}: {score:.4f}")

## Step 7: Generate Sample Captions

In [ ]:
# Test caption generation on sample images
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

# Get random test images
test_images = random.sample(os.listdir('data/images'), 5)

print("Sample Caption Generation:\n")
for img_name in test_images:
    img_path = f'data/images/{img_name}'
    
    # Generate caption with RNN
    !python src/generate_caption.py --image {img_path} --model rnn --checkpoint models/rnn/checkpoint_best.pth > /tmp/rnn_caption.txt
    with open('/tmp/rnn_caption.txt', 'r') as f:
        rnn_caption = f.read().strip()
    
    # Generate caption with Transformer
    !python src/generate_caption.py --image {img_path} --model transformer --checkpoint models/transformer/checkpoint_best.pth > /tmp/transformer_caption.txt
    with open('/tmp/transformer_caption.txt', 'r') as f:
        transformer_caption = f.read().strip()
    
    # Display
    img = Image.open(img_path)
    plt.figure(figsize=(10, 4))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"RNN: {rnn_caption}\nTransformer: {transformer_caption}", fontsize=10)
    plt.show()
    print(f"Image: {img_name}")
    print(f"  RNN: {rnn_caption}")
    print(f"  Transformer: {transformer_caption}\n")

## Step 8: Visualize Attention

In [ ]:
# Generate attention visualization for a sample image
sample_image = test_images[0]
!python src/visualize_attention.py \
    --image data/images/{sample_image} \
    --model rnn \
    --checkpoint models/rnn/checkpoint_best.pth \
    --output outputs/attention_viz.png

# Display attention visualization
from IPython.display import Image as IPImage
display(IPImage('outputs/attention_viz.png'))
print("✅ Attention visualization generated!")

## Step 9: Download Trained Models

In [ ]:
# Zip the trained models
!zip -r trained_models.zip models/ outputs/

# Download
from google.colab import files
files.download('trained_models.zip')

print("✅ Models zipped and ready for download!")
print("\nThe zip file contains:")
print("  - models/rnn/checkpoint_best.pth")
print("  - models/transformer/checkpoint_best.pth")
print("  - outputs/evaluation_results.json")
print("  - outputs/training.log")
print("  - outputs/attention_viz.png")

## Step 10: Launch Interactive Demo (Optional)

In [ ]:
# Launch Gradio demo
!python src/demo.py \
    --rnn-checkpoint models/rnn/checkpoint_best.pth \
    --transformer-checkpoint models/transformer/checkpoint_best.pth \
    --share

print("\n✅ Demo launched! Click the public URL above to access the interactive demo.")

---

## 🎉 Training Complete!

You now have:
- ✅ Trained RNN model with attention
- ✅ Trained Transformer model
- ✅ Evaluation metrics (BLEU, METEOR, CIDEr, ROUGE)
- ✅ Attention visualizations
- ✅ Sample captions
- ✅ Interactive demo

### Next Steps
1. Download the `trained_models.zip` file
2. Extract it locally
3. Use the models for your project report
4. Run the demo locally: `python src/demo.py`

### For Your Report
- Training logs: `outputs/training.log`
- Evaluation results: `outputs/evaluation_results.json`
- Attention visualizations: `outputs/attention_viz.png`
- Model checkpoints: `models/*/checkpoint_best.pth`

---

**Repository:** https://github.com/yichi0812/image-captioning-neural-networks

**Total Training Time:** ~2-3 hours on free T4 GPU
